# Metaverse Avatar

In [42]:
pip install mesa

Note: you may need to restart the kernel to use updated packages.


In [43]:
from mesa import Agent,Model

Here we define the avatar with all its functionalities

In [44]:
class Agent_Avatar(Agent):
    subjects = {}
    preferences = {}

    # First Deliverable
    def __init__(self):
        print('Hello human, I am your avatar, you can configure me to be as similar to you as possible, my creators are Anthony Garcia, Sebastian Arboleda and Katherin Valencia.')

    # Second Deliverable
    def setScheduleOptions(self, subjects, preferences):
        self.subjects = subjects
        self.preferences = preferences

    def getBetterSchedule(self):
        independent, schedule = self.__isIndependent(list(self.subjects.keys()))

        print("============== BETTER WAY TO CHOOSE SUBJECTS ==============")
        score = 0
        for h in schedule:
            subject, dayTime = h
            score += dayTime[1]
            print('Subject: %s, Schedule: %s, Score: %s' % (subject, str(dayTime[0]), dayTime[1]))
        print('TOTAL SCORE: %i' % score)

    def __isIndependent(self, subjs):
        '''Determines if a set of subjects is independent'''
        subjs = self.__orderSubjects(subjs)
        if len(subjs) == 0:
            return False, []
        score, independent = self.__overlap(subjs)
                        
        return True, independent

    def __orderSubjects(self, subjs):
        ordered = []
        for sub in subjs:
            ordered.append([
                sub, self.preferences['subjects'][sub]
            ])
            ordered = sorted(ordered, key=lambda d: d[1], reverse=True)
        subs = []
        for i in ordered:
            subs.append(i[0])
        return subs

    def __overlap(self, subjs):
        return self.__recursiveOverlap([], subjs)

    def __recursiveOverlap(self, actualSchedule, subjs):
        if len(subjs) == 0:
            return 0, actualSchedule
        actualSbj = subjs[0]
        currentScore = 0
        actualScheduleCopy = actualSchedule.copy()

        for i in self.subjects[actualSbj]:
            if not self.__overlapClass(actualSchedule, i):
                newScore = self.__getScore(actualSbj, i)
                schedule = actualScheduleCopy.copy()
                schedule.append((actualSbj, (i, newScore)))
                score, schedule = self.__recursiveOverlap(schedule, subjs[1:])
                if score + newScore > currentScore:
                    actualSchedule = actualScheduleCopy.copy()
                    currentScore = score + newScore
                    actualSchedule = schedule
            score, schedule = self.__recursiveOverlap(actualSchedule, subjs[1:])
            if score > currentScore:
                actualSchedule = actualScheduleCopy.copy()
                currentScore = score
                actualSchedule = schedule
        return currentScore, actualSchedule


    def __overlapClass(self, schedulesList, actualSchedule):
        overlap = True
        for h2 in actualSchedule:
            currentOverlap = False
            for schedule in schedulesList:
                classes = schedule[1][0]
                for h1 in classes:
                    if h1[0] == h2[0]:
                        # Which one starts before
                        if h1[1] <= h2[1]:
                            first, second = h1, h2  
                        else:
                            first, second = h2, h1
                        currentOverlap = currentOverlap or first[2] > second[1]
            overlap = overlap and currentOverlap
        return overlap

    def __getScore(self, subj, schedule):
        subjScore = self.preferences['subjects'][subj]
        scheduleScore = 0.0
        for i in schedule: 
            scheduleScore += self.preferences['days'][i[0]]
        return scheduleScore/len(schedule) + subjScore


Test data for Second Deliverable

In [45]:
subjects = {
    "AI introduction": [[("Monday","0830","1130")], [("Friday", "0830","1130")]] ,
    "Modern SW development processes":  [[("Friday","1800","2100"), ("Saturday", "0800","1100")]],
    "Projects management":  [[("Monday","0600","0900")]],
    "UX Design": [[("Wednesday","1800","2100")]],
    "Karate":  [[("Monday","1630","1800")], [("Wednesday", "1630","1800")], [("Friday", "1630","1800")]],
    "Rest": [[("Friday","0400","0930")]]
}


preferences = {
    "subjects": {
        "AI introduction": 3,
        "Modern SW development processes": 2,
        "Projects management": 1,
        "UX Design": 1,
        "Karate": 2,
        "Rest": 3,
    },
    "days" : {
        "Monday": 2,
        "Tuesday": 4,
        "Wednesday": 4,
        "Thursday": 4,
        "Friday": 3,
        "Saturday": 1,
    },
}

Here we define the Model

In [46]:
class Model_Avatar(Model):
    agent = {}

    def __init__(self):
        self.agent = Agent_Avatar()

    def secondDeliverable(self):
        self.agent.setScheduleOptions(subjects, preferences)
        self.agent.getBetterSchedule()

In [47]:
Avatar = Model_Avatar()

Hello human, I am your avatar, you can configure me to be as similar to you as possible, my creators are Anthony Garcia, Sebastian Arboleda and Katherin Valencia.


## Call for the Search Problem (2nd Deliverable)


In [48]:
Avatar.secondDeliverable()

============== BETTER WAY TO CHOOSE SUBJECTS ==============
Subject: AI introduction, Schedule: [('Monday', '0830', '1130')], Score: 5.0
Subject: Rest, Schedule: [('Friday', '0400', '0930')], Score: 6.0
Subject: Modern SW development processes, Schedule: [('Friday', '1800', '2100'), ('Saturday', '0800', '1100')], Score: 4.0
Subject: Karate, Schedule: [('Wednesday', '1630', '1800')], Score: 6.0
Subject: UX Design, Schedule: [('Wednesday', '1800', '2100')], Score: 5.0
TOTAL SCORE: 26
